# Unsupervised Learning - Dimensionality Reduction and Clustering

## Objective
The goal of this homework assignment is to gain practical experience in performing unsupervised learning tasks, including dimensionality reduction and clustering, using a real dataset.

## Dataset Description
The dataset consists of Wikipedia comments which have been labeled by human raters for toxic behavior. The types of toxicity are:
- toxic
- severe_toxic
- obscene
- threat
- insult
- identity_hate

## Files
- train.csv: Training set, contains comments with their binary labels
- test.csv: Test set, where we predict the toxicity probabilities
- sample_submission.csv: Sample submission file in the correct format

We will perform data exploration, preprocessing, dimensionality reduction, clustering, and visualize the results.

Let's start by importing the necessary libraries.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA, IncrementalPCA, TruncatedSVD
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
import re

## Data Exploration and Preprocessing

First, we'll load the data, explore its characteristics, and preprocess it for further analysis.

In [ ]:
# Load the data (adjust the file paths as needed)
train_df = pd.read_csv('train.csv').sample(n=10000, random_state=42) # Selecting a random subset of 10,000 samples
test_df = pd.read_csv('test.csv')

# Display basic information about the dataset
train_df.info()
train_df.head()

In [ ]:
# Preprocessing steps including text cleaning and vectorization
# Function to clean the text
def clean_text(text):
    # Removing special characters and digits
    text_clean = re.sub(r'[^a-zA-Z]', ' ', text)
    # Converting to lower case
    text_clean = text_clean.lower()
    return text_clean

# Apply cleaning function to the comment text
train_df['cleaned_text'] = train_df['comment_text'].apply(clean_text)

# Vectorizing the text using TfIdfVectorizer
vectorizer = TfidfVectorizer(max_features=3000, stop_words='english')
tfidf_vectors = vectorizer.fit_transform(train_df['cleaned_text'])

print("Shape of the TfIdf matrix:", tfidf_vectors.shape)

## Dimensionality Reduction

We will apply PCA and UMAP to reduce the dimensionality of our dataset.

In [ ]:
# Applying PCA for visualization: Reducing to 2 components
pca_2d = PCA(n_components=2)
pca_2d_result = pca_2d.fit_transform(tfidf_vectors.toarray())

# Display the explained variance for 2 components
print("Explained variance (2 components):", pca_2d.explained_variance_ratio_.sum())

# Optionally, also reduce to 3 components for a different visualization
pca_3d = PCA(n_components=3)
pca_3d_result = pca_3d.fit_transform(tfidf_vectors.toarray())

# Display the explained variance for 3 components
print("Explained variance (3 components):", pca_3d.explained_variance_ratio_.sum())

# Visualization of the PCA result (2D)
plt.figure(figsize=(8, 6))
plt.scatter(pca_2d_result[:, 0], pca_2d_result[:, 1])
plt.title('PCA - 2 Components')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.grid(True)
plt.show()

In [ ]:
import umap

# Applying UMAP to reduce the dataset to 2 components for visualization
umap_2d = umap.UMAP(n_components=2, random_state=42)
umap_2d_result = umap_2d.fit_transform(tfidf_vectors.toarray())

# Visualization of the 2D UMAP result
plt.figure(figsize=(8, 6))
plt.scatter(umap_2d_result[:, 0], umap_2d_result[:, 1])
plt.title('UMAP - 2 Components')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.grid(True)
plt.show()

# Optionally, also reduce to 3 components for a different perspective
umap_3d = umap.UMAP(n_components=3, random_state=42)
umap_3d_result = umap_3d.fit_transform(tfidf_vectors.toarray())

## Clustering

We will perform clustering using K-Means, DBSCAN, and Hierarchical Clustering methods.

In [ ]:
# Applying the Elbow Method to find the optimal number of clusters
inertia = []
k_range = range(1, 11)  # Trying different numbers of clusters
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(tfidf_vectors.toarray())
    inertia.append(kmeans.inertia_)

# Plotting the Elbow graph
plt.figure(figsize=(8, 6))
plt.plot(k_range, inertia, marker='o')
plt.title('Elbow Method for Determining Optimal Number of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.xticks(k_range)
plt.grid(True)
plt.show()

# Choose the number of clusters where the elbow occurs
optimal_k = 4  # Replace with the appropriate number as per the elbow plot

# Performing K-Means clustering with the optimal number of clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
kmeans_clusters = kmeans.fit_predict(tfidf_vectors.toarray())

# The variable 'kmeans_clusters' now holds the cluster assignments for each data point

In [ ]:
# Applying DBSCAN clustering
# Note: The parameters 'eps' and 'min_samples' may need to be adjusted based on your dataset
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_clusters = dbscan.fit_predict(tfidf_vectors.toarray())

# dbscan_clusters now contains the cluster labels for each data point
# Noise points are labeled as -1

In [ ]:
# Applying Hierarchical Clustering
# 'optimal_k' should be the number of clusters you determined to be optimal (e.g., from the Elbow Method in K-Means)
optimal_k = 4  # Replace with your chosen number of clusters
hierarchical = AgglomerativeClustering(n_clusters=optimal_k)
hierarchical_clusters = hierarchical.fit_predict(tfidf_vectors.toarray())  # Replace with your vectorized data

# hierarchical_clusters now contains the cluster assignments for each data point

## Visualization

Finally, we will visualize the clusters created by each clustering method.

In [ ]:
# Visualization for K-Means Clusters
plt.figure(figsize=(8, 6))
plt.scatter(pca_2d_result[:, 0], pca_2d_result[:, 1], c=kmeans_clusters)
plt.title('K-Means Clustering Visualization')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.colorbar(label='Cluster Label')
plt.show()

# Visualization for DBSCAN Clusters
plt.figure(figsize=(8, 6))
plt.scatter(pca_2d_result[:, 0], pca_2d_result[:, 1], c=dbscan_clusters)
plt.title('DBSCAN Clustering Visualization')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.colorbar(label='Cluster Label')
plt.show()

# Visualization for Hierarchical Clustering
plt.figure(figsize=(8, 6))
plt.scatter(pca_2d_result[:, 0], pca_2d_result[:, 1], c=hierarchical_clusters)
plt.title('Hierarchical Clustering Visualization')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.colorbar(label='Cluster Label')
plt.show()